In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices()
tf.config.experimental.set_memory_growth(physical_devices[3], True)
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import layers

In [2]:
AUTOTUNE  = tf.data.experimental.AUTOTUNE
INPUT_SHAPE = (224, 224, 3)
BATCH_SIZE = 8

In [3]:
# Defining the model
def get_model():
    x_input = Input(INPUT_SHAPE)
    pretrained_model = MobileNetV2(input_shape=INPUT_SHAPE, 
                                   alpha=1.0, 
                                   include_top=False,
                                   weights='imagenet',
                                   input_tensor=x_input) 
    pretrained_model.trainable = True
    x = tf.reduce_mean(pretrained_model.outputs[0], axis = [1, 2])   #Global Average Pooling
    x = Flatten()(x)
    x = Dense(512, kernel_initializer = "he_normal")(x)
    x = tf.nn.relu(x)
    x = Dropout(0.2)(x)
    x = Dense(256, kernel_initializer = "he_normal")(x)
    x = tf.nn.relu(x)
    x = Dense(128, kernel_initializer = "he_normal")(x)
    x = tf.nn.relu(x)
    x = Dense(64, kernel_initializer ="he_normal")(x)
    x = tf.nn.relu(x)
    x = Dense(64, kernel_initializer ="he_normal")(x)
    x = tf.nn.relu(x)
    x = Dense(16, kernel_initializer ="he_normal")(x)
    x = tf.nn.relu(x)
    x = Dense(8, kernel_initializer ="he_normal")(x)
    x = tf.nn.relu(x)
    x = Dense(1, kernel_initializer ="he_normal")(x)
    x = tf.nn.sigmoid(x)
    model = Model(x_input, x)
    return model

model = get_model()
                                  

In [4]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [5]:
#Importing the files from disk space
train_dir = str(os.getcwd()) + "/New Masks Dataset/Train/"
val_dir = str(os.getcwd()) + "/New Masks Dataset/Validation/"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_dir, 
                                                               class_names=["WithMask", "WithoutMask"],
                                                               color_mode='rgb',
                                                               batch_size=BATCH_SIZE, 
                                                               image_size=(INPUT_SHAPE[0],INPUT_SHAPE[1]),
                                                               shuffle=True,
                                                               interpolation='bilinear')


val_ds = tf.keras.preprocessing.image_dataset_from_directory(val_dir, 
                                                               class_names=["WithMask", "WithoutMask"],
                                                               color_mode='rgb',
                                                               batch_size=BATCH_SIZE, 
                                                               image_size=(INPUT_SHAPE[0],INPUT_SHAPE[1]),
                                                               shuffle=True,
                                                               interpolation='bilinear')

#Rescaling the image between in range [0.0, 1.0]

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


Found 10000 files belonging to 2 classes.
Found 800 files belonging to 2 classes.


In [6]:
train_ds = train_ds.take(625)

In [7]:
def configure_for_performance(ds, train="no"):
    if train=="no":
        ds = ds.cache(filename="val_cache")
    else:
        ds = ds.cache(filename = f"train_{int(train)}_cache")
    ds = ds.shuffle(buffer_size=40)
#     ds = ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds
    
train_ds = configure_for_performance(train_ds, train=1)
val_ds = configure_for_performance(val_ds)

In [8]:
model.trainable = True
model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.0000025), loss = "binary_crossentropy", metrics = ['binary_accuracy'])

callbacks = [tf.keras.callbacks.ModelCheckpoint(str(os.getcwd()) +"/weights.hdf5",
                                                monitor='val_binary_accuracy',
                                                verbose=0, 
                                                save_best_only=True,
                                                save_weights_only=True, 
                                                mode='max',
                                                save_freq='epoch')]



In [9]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [10]:
model.fit(train_ds,
          batch_size =BATCH_SIZE,
          epochs = 50, 
          validation_data = val_ds,
          callbacks = callbacks)

Epoch 1/50
625/625 [==============================] - 61s 98ms/step - loss: 0.5845 - binary_accuracy: 0.6360 - val_loss: 0.4792 - val_binary_accuracy: 0.8062
Epoch 2/50
625/625 [==============================] - 59s 95ms/step - loss: 0.3740 - binary_accuracy: 0.9022 - val_loss: 0.2938 - val_binary_accuracy: 0.9325
Epoch 3/50
625/625 [==============================] - 59s 95ms/step - loss: 0.2377 - binary_accuracy: 0.9466 - val_loss: 0.1557 - val_binary_accuracy: 0.9812
Epoch 4/50
625/625 [==============================] - 59s 94ms/step - loss: 0.1549 - binary_accuracy: 0.9570 - val_loss: 0.0883 - val_binary_accuracy: 0.9850
Epoch 5/50
625/625 [==============================] - 59s 94ms/step - loss: 0.1109 - binary_accuracy: 0.9692 - val_loss: 0.0524 - val_binary_accuracy: 0.9925
Epoch 6/50
625/625 [==============================] - 59s 94ms/step - loss: 0.0799 - binary_accuracy: 0.9770 - val_loss: 0.0346 - val_binary_accuracy: 0.9975
Epoch 7/50
625/625 [==============================] 

KeyboardInterrupt: 

In [ ]:
for img, lab in val_ds.take(4):
    plt.imshow(img[0], cmap = "gray")
    plt.show()
    print(lab[0])
    print(model(img, training = False)[0])